**Each of the four of us used this identical preeliminary cleaning steps for our 6 months of data. Which was then compiled into one giant dataset called total.csv**

In [1]:
import pandas as pd
import glob

In [2]:
def get_dataframe(path):
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [3]:
tix_jul18_dec18 = get_dataframe('./Data/parking_jul18_dec18')

C:\Users\Boris\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
tix_jul18_dec18.shape

(687264, 31)

In [5]:
tix_jul18_dec18.columns

Index(['OBJECTID', 'TICKET_NUMBER', 'VIOLATION_TYPE_DESC', 'ISSUE_DATE',
       'ISSUE_TIME', 'ISSUING_AGENCY_CODE', 'ISSUING_AGENCY_NAME',
       'ISSUING_AGENCY_SHORT', 'VIOLATION_CODE', 'VIOLATION_PROC_DESC',
       'LOCATION', 'PLATE_STATE', 'VEHICLE_TYPE', 'MULTI_OWNER_NUMBER',
       'DISPOSITION_CODE', 'DISPOSITION_TYPE', 'DISPOSITION_DESC',
       'DISPOSITION_DATE', 'FINE_AMOUNT', 'TOTAL_PAID', 'PENALTY_1',
       'PENALTY_2', 'PENALTY_3', 'PENALTY_4', 'PENALTY_5', 'XCOORD', 'YCOORD',
       'LATITUDE', 'LONGITUDE', 'MAR_ID', 'GIS_LAST_MOD_DTTM'],
      dtype='object')

In [6]:
tix_jul18_dec18.head(1)

,OBJECTID,TICKET_NUMBER,VIOLATION_TYPE_DESC,ISSUE_DATE,ISSUE_TIME,ISSUING_AGENCY_CODE,ISSUING_AGENCY_NAME,ISSUING_AGENCY_SHORT,VIOLATION_CODE,VIOLATION_PROC_DESC,...,PENALTY_2,PENALTY_3,PENALTY_4,PENALTY_5,XCOORD,YCOORD,LATITUDE,LONGITUDE,MAR_ID,GIS_LAST_MOD_DTTM
0,60332296,260549310.0,P,2018/08/11 04:00:00+00,09:00 AM,54.0,ST.ELZBETH HOSPITAL SECURITY GUARDS,SEH,P001,PARK ABREAST OF ANOTHER VEHICLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019/05/29 04:16:08+00


In [8]:
tix_jul18_dec18['DISPOSITION_TYPE'].value_counts()

Other               400993
Liable - System     199429
Dismissed            44266
Liable               40776
Void                  1001
Continued              730
Administrative          41
Liable - Traffic        28
Name: DISPOSITION_TYPE, dtype: int64

In [9]:
tix_jul18_dec18.isnull().sum()

OBJECTID                     0
TICKET_NUMBER           202534
VIOLATION_TYPE_DESC     202534
ISSUE_DATE                   0
ISSUE_TIME                 505
ISSUING_AGENCY_CODE        170
ISSUING_AGENCY_NAME        170
ISSUING_AGENCY_SHORT       170
VIOLATION_CODE               0
VIOLATION_PROC_DESC      24259
LOCATION                   119
PLATE_STATE                  0
VEHICLE_TYPE            687263
MULTI_OWNER_NUMBER      653466
DISPOSITION_CODE        400257
DISPOSITION_TYPE             0
DISPOSITION_DESC        400257
DISPOSITION_DATE        400257
FINE_AMOUNT              24908
TOTAL_PAID                   0
PENALTY_1               403913
PENALTY_2               687264
PENALTY_3               573349
PENALTY_4               687146
PENALTY_5               687264
XCOORD                   21237
YCOORD                   21237
LATITUDE                 21237
LONGITUDE                21237
MAR_ID                   21237
GIS_LAST_MOD_DTTM            0
dtype: int64

In [10]:
#columns with >80% nulls
tix_jul18_dec18.drop(columns=['DISPOSITION_CODE','DISPOSITION_DATE','PENALTY_1',
                             'PENALTY_2','PENALTY_3','PENALTY_4','PENALTY_5',
                             'VEHICLE_TYPE','MULTI_OWNER_NUMBER','DISPOSITION_DESC',
                            'TICKET_NUMBER', 'VIOLATION_TYPE_DESC'],inplace=True)

In [11]:
# other columns to drop
# PLATE_STATE - empty column, no values
# GIS_LAST_MOD_DTTM - i think this is the date dataset was uploaded into database

# Empty column, no values
tix_jul18_dec18.drop(columns=['PLATE_STATE', 'GIS_LAST_MOD_DTTM'],inplace=True)

In [12]:
#lowercase column names
tix_jul18_dec18.columns = map(str.lower, tix_jul18_dec18.columns)

In [13]:
#drop issue_time nulls
tix_jul18_dec18 = tix_jul18_dec18[tix_jul18_dec18['issue_time'].notna()]

#drop issue_time rows that don't have proper format
tix_jul18_dec18 = tix_jul18_dec18[tix_jul18_dec18['issue_time'].str.len()==8]

In [14]:
#remove characters after space in issue date
tix_jul18_dec18['issue_date'] = tix_jul18_dec18['issue_date'].str.split().str[0]
tix_jul18_dec18['issue_date'].head()

0    2018/08/11
1    2018/08/09
3    2018/08/14
4    2018/08/23
5    2018/08/23
Name: issue_date, dtype: object

In [15]:
#create new column date plus time
tix_jul18_dec18['issue_datetime']= tix_jul18_dec18['issue_date']+' '+tix_jul18_dec18['issue_time']
tix_jul18_dec18['issue_datetime'].head(2)

0    2018/08/11 09:00 AM
1    2018/08/09 09:15 AM
Name: issue_datetime, dtype: object

In [16]:
#convert to date time format
tix_jul18_dec18['issue_datetime'] = tix_jul18_dec18['issue_datetime'].astype('datetime64[ns]')
tix_jul18_dec18['issue_datetime'].head(1)

0   2018-08-11 09:00:00
Name: issue_datetime, dtype: datetime64[ns]

In [17]:
#add columns for year, month, date, dayofweek, hour, minute
tix_jul18_dec18['year'] = tix_jul18_dec18['issue_datetime'].dt.year
tix_jul18_dec18['month'] = tix_jul18_dec18['issue_datetime'].dt.month
tix_jul18_dec18['date'] = tix_jul18_dec18['issue_datetime'].dt.day
tix_jul18_dec18['day'] = tix_jul18_dec18['issue_datetime'].dt.dayofweek
tix_jul18_dec18['hour'] = tix_jul18_dec18['issue_datetime'].dt.hour
tix_jul18_dec18['minute'] = tix_jul18_dec18['issue_datetime'].dt.minute

In [18]:
#drop unused columns
tix_jul18_dec18.drop(columns=['issue_date','issue_time','issue_datetime'],inplace=True)

# dropping rows with NaN values in these columns
tix_jul18_dec18.dropna(axis=0, subset=['xcoord','ycoord','latitude','longitude','mar_id'], inplace=True)

# Dropping mar_id column
tix_jul18_dec18.drop(columns=['mar_id'],inplace=True)

# dropping rows with NaN values in these columns
tix_jul18_dec18.dropna(axis=0, subset=['violation_proc_desc','fine_amount'], inplace=True)

In [19]:
tix_jul18_dec18.isnull().sum()

objectid                0
issuing_agency_code     0
issuing_agency_name     0
issuing_agency_short    0
violation_code          0
violation_proc_desc     0
location                0
disposition_type        0
fine_amount             0
total_paid              0
xcoord                  0
ycoord                  0
latitude                0
longitude               0
year                    0
month                   0
date                    0
day                     0
hour                    0
minute                  0
dtype: int64

In [20]:
tix_jul18_dec18.shape

(641385, 20)

In [21]:
# Saving cleaned dataset for my months
# tix_jul18_dec18.to_csv('data/boris_months.csv', index=False)